# Visualization of Canada cumulative cases prediction
We will use the AR Model to predict the cumulative number of cases in Canada in the next 30 days on 7 consecutive dates.

In [1]:
from __future__ import print_function
import numpy as np, pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
import datetime
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import pmdarima as pm
import plotly.graph_objects as go
import scipy.stats as ss
from pandas.tseries.offsets import Day
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
# functions
def ar_model(cases):
    model_ar=pm.auto_arima(cases,trace = False,error_action='ignore', start_p=0,start_q=0,max_p=4,max_q=0,
                   stepwise=False,seasonal=False)
    model_ar.fit(cases)
    return model_ar

def Hex_to_RGB(hex):
    hex = hex.lstrip('#')
    r = int(hex[0:2],16)
    g = int(hex[2:4],16)
    b = int(hex[4:6], 16)
    rgb = 'rgba('+str(r)+','+str(g)+','+str(b)+','
    return rgb

#color_sequence=["rgba(255,0,0,", "rgba(0,255,0,", "rgba(0,0,255,","rgba(255,165,0,","rgba(255,255,0,","rgba(0,127,255,", "rgba(139,0,255,"]
#红 绿 蓝 橙 黄 青 紫

def plotting(start_fc1,opacity,Prediction_1,Prediction_2,Prediction_3,Prediction_4,Prediction_5,Prediction_6,Prediction_7):
    cases = pd.read_csv(r'cases_timeseries_canada.csv', parse_dates=['date'], index_col='date')
    cases=cases.drop(["new_cases"],axis=1)
    cases_t=[]
    cases_p=[]
    index_of_fc=[]
    
    start_fc2=pd.to_datetime(start_fc1)+6*Day()
    
    for date in pd.date_range(pd.to_datetime(start_fc1)-Day(),pd.to_datetime(start_fc2)-Day()):
        cases_t.append(cases["2020-01-25":date])
    for date in pd.date_range(start_fc1,start_fc2):
        cases_p.append(cases[date:date+30*Day()])
    for date in pd.date_range(start_fc1,start_fc2):
        index_of_fc.append(pd.date_range(date,date+30*Day()))

    cases_tt=cases["2020-03-29":pd.to_datetime(start_fc2)-Day()]
    index_of_fc_total= pd.date_range(start_fc1,pd.to_datetime(start_fc2)+30*Day())

    n_periods={}
    for i in range(len(cases_t)):
        n_periods[i]=len(cases_p[i])
    
    model_ar={}
    fc_ar={}
    confint_ar={}
    fc_series_ar={}
    lower_series_ar={}
    upper_series_ar={}
    for i in range(len(cases_t)):
        model_ar[i]=ar_model(cases_t[i])
        fc_ar[i], confint_ar[i] = model_ar[i].predict(n_periods=n_periods[i], return_conf_int=True)
        fc_series_ar[i] = pd.Series(fc_ar[i], index=index_of_fc[i])
        lower_series_ar[i] = pd.Series(confint_ar[i][:, 0], index=index_of_fc[i])
        upper_series_ar[i] = pd.Series(confint_ar[i][:, 1], index=index_of_fc[i])
    
    colors=[Hex_to_RGB(Prediction_1),Hex_to_RGB(Prediction_2),Hex_to_RGB(Prediction_3),Hex_to_RGB(Prediction_4),Hex_to_RGB(Prediction_5),Hex_to_RGB(Prediction_6),Hex_to_RGB(Prediction_7)]
    fig=go.Figure()
    for i in range(len(cases_t)):
        fig.add_trace(go.Scatter(x=index_of_fc[i], y=fc_series_ar[i],mode='lines',line = dict(color=colors[i]+"1)"),name="Prediction "+str(i+1)))
        fig.add_trace(go.Scatter(x=index_of_fc[i], y=lower_series_ar[i],mode='lines',line = dict(color=colors[i]+str(opacity)+ ")"),showlegend=False))
        fig.add_trace(go.Scatter(x=index_of_fc[i], y=upper_series_ar[i],mode='lines',line = dict(color=colors[i]+str(opacity)+ ")"),showlegend=False,fill="tonextx")) 
    fig.add_trace(go.Scatter(x=cases_tt.index, y=cases_tt["cumulative_cases"],mode='lines+markers',line = dict(color="black"),name="Real Data for Cumulative Cases"))
    fig.update_layout(title="Cumulative Cases AR Model Prediction",xaxis_title="Date",yaxis_title="Cumulative Cases",yaxis2=dict(title="RMSE", overlaying='y', side="right"),legend=dict(x=0,y=1,traceorder="normal"))
    fig.show()


In [3]:
w=interactive(plotting,start_fc1=widgets.DatePicker(
    description='Pick a Date',
    disabled=False,
    value = datetime.date(2020,4,1)
   
),opacity=(0,1,0.1),
Prediction_1=widgets.ColorPicker(
    concise=False,
    description='Prediction 1',
    value='#ffa500',
    disabled=False
),
Prediction_2=widgets.ColorPicker(
    concise=False,
    description='Prediction 2',
    value='#ffa500',
    disabled=False
),
Prediction_3=widgets.ColorPicker(
    concise=False,
    description='Prediction 3',
    value='#ffa500',
    disabled=False
),
Prediction_4=widgets.ColorPicker(
    concise=False,
    description='Prediction 4',
    value='#ffa500',
    disabled=False
),
Prediction_5=widgets.ColorPicker(
    concise=False,
    description='Prediction 5',
    value='#ffa500',
    disabled=False
),
Prediction_6=widgets.ColorPicker(
    concise=False,
    description='Prediction 6',
    value='#ffa500',
    disabled=False
),
Prediction_7=widgets.ColorPicker(
    concise=False,
    description='Prediction 7',
    value='#ffa500',
    disabled=False
)         )

## Three things you can do with the plot below:
* You can decide the start date of the 7-day continuous predictions by the **date picker** below(the date must be between 2020-04-01 and 2021-05-01).
* You can change the overall opacity of the confidence intervals of the 7 predictions by dragging the **slider** below.
* You can change the color of the confidence intervals for each prediction through the **color pickers** below.

In [4]:
#show the widgets and plot
display(w)

interactive(children=(DatePicker(value=datetime.date(2020, 4, 1), description='Pick a Date'), FloatSlider(valu…

In [5]:
%load_ext watermark
%watermark -v -m -p numpy,statsmodels,matplotlib,pmdarima,plotly,scipy,ipywidgets,IPython,datetime,watermark 

Python implementation: CPython
Python version       : 3.7.6
IPython version      : 7.12.0

numpy      : 1.19.5
statsmodels: 0.12.2
matplotlib : 3.1.3
pmdarima   : 1.8.2
plotly     : 5.3.1
scipy      : 1.5.4
ipywidgets : 7.6.5
IPython    : 7.12.0
datetime   : 4.3
watermark  : 2.2.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 61 Stepping 4, GenuineIntel
CPU cores   : 4
Architecture: 64bit

